#Paquetes necesarios

# TRAIN YOLO

In [6]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
results = model.train(data='platedataset/data.yaml', epochs=3)

Ultralytics YOLOv8.0.203 🚀 Python-3.11.4 torch-2.1.0 CPU (Intel Core(TM) i7-9750H 2.60GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=platedataset/data.yaml, epochs=3, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, opti

In [1]:
class PlateDetector:
    def detect(self, img):
        pass

class PlateMatcher:
    def match(self, text : str) -> bool:
        pass

class TextProcessor:
    def process(self, text : str) -> str:
        pass

class PlateExtractor:
    def __init__(self, matcher : PlateMatcher, processor : TextProcessor):
        self.matcher = matcher
        self.processor = processor

    def extract(self, img):
        pass

In [2]:
import re

class LengthPlateMatcher(PlateMatcher):
    def match(self, text : str) -> bool:
        if len(text) == 7: return True
        return False

class RegexPlateMatcher(PlateMatcher):
    def match(self, text : str) -> bool:
        pattern = re.compile("^\d{4}[A-Z]{3}$")
        # pattern = re.compile("^[0-9]{4}([B-D]|[F-H]|[J-N]|[P-T]|[V-Z]){3}$")
        if pattern.match(text):
            return True
        else:
            return False

In [3]:
import easyocr
import pytesseract
from keras_ocr.recognition import Recognizer
from keras_ocr.detection import Detector

class TesseractPlateExtractor(PlateExtractor):
    def extract(self, img):
        pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/tesseract'
        text = pytesseract.image_to_string(img)
        print(text)
        matcher = RegexPlateMatcher()
        if matcher.match(text):
            text = self.processor.process(text)
            return text
        
class KerasOCRPlateExtractor(PlateExtractor):
    def extract(self, img):
        detector = Detector()
        recognizer = Recognizer()
        pipeline = pipeline.Pipeline(detector=detector, recognizer=recognizer)
        prediction_groups = pipeline.recognize([img])
        for group in prediction_groups:
            for word in group:
                text = word[0]
                text = self.processor.process(text)
                if(self.matcher.match(text)):
                    return text

class EasyOCRPlateExtractor(PlateExtractor):
    def extract(self, img):
        reader = easyocr.Reader(['es'], detector='dbnet18') 
        result = reader.readtext(img, batch_size=4)
        if(len(result) == 0): return ""
        finaltext = ""
        for x in reversed(result):
            text = x[1]
            text = self.processor.process(text)
            finaltext += str(text)
        if(self.matcher.match(finaltext)):
            return finaltext

In [4]:
class BasicTextProcessor(TextProcessor):
    def process(self, text : str) -> str:
        text = text.replace(" ", "")
        text = text.replace("-", "")
        return text

In [5]:
from ultralytics import YOLO

class YOLOPlateDetector(PlateDetector):
    def __init__(self, model, category = 0):
        self.model = YOLO(model).cuda()
        self.category = category

    def detect(self, img):
        results = self.model(img)
        plates = []
        for x in results:
            boxes = x.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                category = int(box.cls[0])
                if category == 0:
                    plates.append((x1, y1, x2, y2, img[y1:y2, x1:x2]))
        return plates

In [6]:
import torch
torch.cuda.is_available()

True

In [7]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'

In [8]:
import cv2
import time

image = cv2.imread('./plates/coches2.jpg')
processor = BasicTextProcessor()
# matcher = LengthPlateMatcher()
matcher = RegexPlateMatcher()
detector = YOLOPlateDetector(model='plate_recognizer.pt')
extractor = EasyOCRPlateExtractor(matcher, processor)
start = time.time()
plates = detector.detect(image)
print("plate detection time:", time.time() - start, "seconds")

# Dibujar rectángulos alrededor de las placas detectadas
for plate in plates:
    if len(plate) == 5:  # Asegurarse de que haya cinco valores en la tupla
        x1, y1, x2, y2, img = plate
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        start = time.time()
        detection = extractor.extract(img)
        print("easyocr reading time:", time.time() - start, "seconds")
        if detection == None or detection == '': detection = "unknown plate"
        cv2.putText(image, str(detection), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Mostrar la imagen con los rectángulos dibujados
cv2.imshow('Plates Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 384x640 1 License_Plate, 55.2ms
Speed: 4.3ms preprocess, 55.2ms inference, 158.4ms postprocess per image at shape (1, 3, 384, 640)


plate detection time: 0.7690033912658691 seconds
easyocr reading time: 2.8750483989715576 seconds


In [9]:
import cv2

cap = cv2.VideoCapture('videos/license_plates_fps.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) / 2)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) / 2)

processor = BasicTextProcessor()
# matcher = LengthPlateMatcher()
matcher = RegexPlateMatcher()
detector = YOLOPlateDetector(model='plate_recognizer.pt')
extractor = EasyOCRPlateExtractor(matcher, processor)

while True:
    ret, frame = cap.read()
    if not ret: break

    frame = cv2.resize(frame, (width, height))
    plates = detector.detect(frame)

    # Dibujar rectángulos alrededor de las placas detectadas
    for plate in plates:
        if len(plate) == 5:  # Asegurarse de que haya cuatro valores en la tupla
            x1, y1, x2, y2, img = plate
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            detection = extractor.extract(img)
            if detection == None or detection == '': detection = "unknown plate"
            cv2.putText(frame, detection, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('Plates', frame)
    if cv2.waitKey(20) == 27: 
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 2 License_Plates, 25.1ms
Speed: 4.8ms preprocess, 25.1ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 17.6ms
Speed: 3.9ms preprocess, 17.6ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 12.9ms
Speed: 3.4ms preprocess, 12.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 10.3ms
Speed: 2.0ms preprocess, 10.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
